In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
factor_data=pd.read_excel('factor_pricing_data_monthly.xlsx', sheet_name='factors (excess returns)')
factor_data = factor_data.sort_values('Date').reset_index(drop=True)


## The Factors

1. Analyze the factors, similar to how you analyzed the three Fama-French factors in Homework 4.

In [15]:
factor_data= factor_data[['MKT','SMB','HML','RMW','CMA','UMD']]
def performance_stats(series):
    mean = series.mean()
    vol = series.std()
    sharpe = mean / vol
    var_05 = np.percentile(series, 5)
    return pd.Series({
        'Mean': mean,
        'Volatility': vol,
        'Sharpe': sharpe,
        'VaR(0.05)': var_05
    })

factors = ['MKT','SMB','HML','RMW','CMA','UMD']
results = {}
for factor in factors:
    results[factor] = pd.DataFrame({
        'Performance Statistics': performance_stats(factor_data[factor])
    })
    print(f"\nPerformance Statistics for {factor}")
    display(results[factor].round(4))


Performance Statistics for MKT


,Performance Statistics
Mean,0.0073
Volatility,0.0451
Sharpe,0.1619
VaR(0.05),-0.0724



Performance Statistics for SMB


,Performance Statistics
Mean,0.0005
Volatility,0.0292
Sharpe,0.0174
VaR(0.05),-0.0431



Performance Statistics for HML


...

,Performance Statistics
Mean,0.0037
Volatility,0.0239
Sharpe,0.1533
VaR(0.05),-0.0285



Performance Statistics for CMA


,Performance Statistics
Mean,0.0024
Volatility,0.0209
Sharpe,0.1127
VaR(0.05),-0.0278



Performance Statistics for UMD


,Performance Statistics
Mean,0.0050
Volatility,0.0443
Sharpe,0.1135
VaR(0.05),-0.0673


2. Based on the factor statistics above, answer the following.  
- Does each factor have a positive risk premium (positive expected excess return)?  
- How have the factors performed since the time of the case, (2015-present)?

3. Report the correlation matrix across the six factors.  
- Does the construction method succeed in keeping correlations small?  
- Fama and French say that HML is somewhat redundant in their 5-factor model. Does this seem to be the case?  

In [16]:
corr_full = factor_data[factors].corr()
print("Correlation matrix (Full sample):")
display(corr_full)

Correlation matrix (Full sample):


,MKT,SMB,HML,RMW,CMA,UMD
MKT,1.000000,0.226997,-0.207918,-0.250639,-0.346542,-0.179352
SMB,0.226997,1.000000,-0.021819,-0.411946,-0.051099,-0.060940
HML,-0.207918,-0.021819,1.000000,0.219401,0.676727,-0.215523
RMW,-0.250639,-0.411946,0.219401,1.000000,0.138566,0.076694
CMA,-0.346542,-0.051099,0.676727,0.138566,1.000000,0.000094
UMD,-0.179352,-0.060940,-0.215523,0.076694,0.000094,1.000000


4. Report the tangency weights for a portfolio of these 6 factors.  
- Which factors seem most important? And Least?  
- Are the factors with low mean returns still useful?  
- Re-do the tangency portfolio, but this time only include MKT, SMB, HML, and UMD. Which factors get high/low tangency weights now?  
- What do you conclude about the importance or unimportance of these styles?  



In [17]:
def tangency_weights(returns, cov_mat = 1):
    cov = returns.cov()
    if cov_mat == 1:
        cov_used = cov
    else:
        cov_diag = np.diag(np.diag(cov))
        cov_used = cov_mat * cov + (1 - cov_mat) * cov_diag
    cov_inv = np.linalg.inv(cov_used * 12)

    # --- Mean returns (annualized) ---
    mu = returns.mean() * 12

    # --- Tangency portfolio computation ---
    w_unnormalized = cov_inv @ mu
    w_tangency = w_unnormalized / np.sum(w_unnormalized)

    # --- Output as DataFrame ---
    tangency_wts = pd.DataFrame(w_tangency, index=returns.columns, columns=['Tangency Weights'])
    return tangency_wts.round(4)

In [18]:
returns = factor_data[factors]

tangency_wts = tangency_weights(returns)
display(tangency_wts)

,Tangency Weights
MKT,0.2186
SMB,0.0668
HML,-0.0212
RMW,0.3018
CMA,0.3214
UMD,0.1125


In [19]:
#expected annualized return, volatility, and Sharpe ratio of that tangency portfolio
cov = returns.cov() * 12
mu = returns.mean() * 12
w = tangency_wts['Tangency Weights'].values

port_return = w.T @ mu
port_vol = np.sqrt(w.T @ cov @ w)
sharpe = port_return / port_vol

print(f"Tangency Portfolio: Return = {port_return:.4f}, Vol = {port_vol:.4f}, Sharpe = {sharpe:.4f}")


Tangency Portfolio: Return = 0.0482, Vol = 0.0401, Sharpe = 1.2013


In [20]:
#Redone tangency portfolio only MKT, SMB, HML, and UMD
factors_reduced = ['MKT','SMB','HML','UMD']
returns_reduced = factor_data[factors_reduced]
tangency_wts_reduced = tangency_weights(returns_reduced)
display(tangency_wts_reduced)

,Tangency Weights
MKT,0.3765
SMB,-0.0512
HML,0.3653
UMD,0.3094


## Testing Modern LPMs 
